In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import sys


sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification')
sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification/src')

from llm_lexical_simplifier import LLMLexicalSimplifier
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from tqdm import tqdm

# LLM based

Install this and the restart the session. This is a workaround for a known bug in flash-attn. Do not re-run this cell after restarting the session.

In [ ]:
#!pip install accelerate flash-attn

In [4]:
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    #device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
    # Flash attention is only supported starting Ampere architecture
    # So we cant use it on free-tier GPUs
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
generation_args = {
    "max_new_tokens": 100,
    "return_full_text": False,
    "do_sample": False,
}

# Define PAD Token = EOS Token
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [9]:
messages = [
    {"role": "user", "content": "Der Maschinenbau hat in Europa durch die Bildung der EU eine starke Erleichterung erhalten. Die vereinfachte Version des vorigen Satzes ist: Der Maschinenbau hat in Europa durch die Bildung der EU eine starke [MASK] erhalten. Results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["Vereinfachung", "Entspannung", "Begünstigung", "Unterstützung", "Rückenwind,  "Förderung", "Rückhalt", "Aufwind", "Antrieb", "Erleichterung"]'},
    {"role": "user", "content": "Die EU-Renaturierungsverordnung galt eigentlich bereits als durchgebracht; das EU-Parlament hatte zugestimmt und die Staaten ihren grundsätzlichen Sanktus signalisiert. Die vereinfachte Version des vorigen Satzes ist: Die EU-Renaturierungsverordnung galt eigentlich bereits als durchgebracht; das EU-Parlament hatte zugestimmt und die Staaten ihren grundsätzlichen [MASK] signalisiert. Ten results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["Zustimmung", "Einverständnis", "Okay", "Segen", "Billigung, Bestätigung, Bejahung, Zuspruch, Akzeptanz, Befürwortung"]'},
]

llm_ls = LLMLexicalSimplifier(model=model, tokenizer=tokenizer,
                              pattern='{original_sentence}. Die vereinfachte Version des vorigen Satzes ist: {sentence_with_complex_word_masked}. Ten results as ordered, AST parsable Python list:',
                              exemplars=messages, mask_token='[MASK]', generation_args=generation_args)

Using cuda for model inference.
Using mask token: "[MASK]".


In [10]:
llm_output = llm_ls.generate_substitutions_for([['Die politische Lage in Syrien ist heikel.','heikel']])
llm_output

['kompliziert', 'instabil', 'bedrohlich', 'kränkend', 'verschlechtert']

In [ ]:
from benchmark_suite import BenchmarkSuite
from language import Language
suite = BenchmarkSuite(llm_ls, {
        Language.DE: {'pattern': '{original_sentence}. Die vereinfachte Version des vorigen Satzes ist: '
                                 '{sentence_with_complex_word_masked}. Ten results as ordered, AST parsable '
                                 'Python list:',
                      'exemplars': messages}})
suite.run()

Benchmarking model on DE ...
Benchmarking model on GermanEvalDataProvider...


Benchmarking:   0%|          | 0/1040 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Benchmarking:   0%|          | 3/1040 [00:12<1:28:27,  5.12s/it]

Failed to parse the output from the LLM: The provided string is not a valid list representation:  In der Legislaturperiode 1998–2002 wurden unter anderem die Ökosteuer (allerdings in einer gegenüber grünen Vorstellungen reduzierten Form), einige Reformen des Staatsbürgerschaftsrechts bezüglich der [Erleichterung] von Einwanderung, die Möglichkeit eingetragener Lebenspartnerschaften und der langfristige Ausstieg aus der Atomenergie verabschiedet.Returning empty list.


Benchmarking:   0%|          | 4/1040 [00:15<1:14:05,  4.29s/it]

Failed to parse the output from the LLM: The provided string is not a valid list representation:  AtmosphäreReturning empty list.


Benchmarking:   0%|          | 5/1040 [00:19<1:08:10,  3.95s/it]

Failed to parse the output from the LLM: The provided string is not a valid list representation: Returning empty list.


#BERT based

In [ ]:
from simple_bert_lexical_simplifier import SimpleBertLexicalSimplifier
bert_ls = SimpleBertLexicalSimplifier(model, tokenizer, bert_pattern, None)

Using cuda for model inference.
No exemplars provided, using zero-shot mode.


In [ ]:
from benchmark_suite import BenchmarkSuite
from language import Language
suite = BenchmarkSuite(bert_ls, {Language.EN: '{original_sentence} The simpler version of the previous sentence is: {sentence_with_complex_word_masked}',
                                 Language.DE: '{original_sentence} Die vereinfachte Version des vorigen Satzes ist: {sentence_with_complex_word_masked}'})
suite.run()

KeyboardInterrupt: 